## This code is used to:
1. Prepare dataset from CSV file (from Google Sheets)
2. Preprocess dataset by replacing entities with tags and/or blacking them out

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 
from os import listdir
import os
from tqdm import tqdm_notebook

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import gensim
from gensim.models import KeyedVectors

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.pipeline import merge_entities
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

Prepare Aadhar Dataset

In [4]:
# read data from CSV files in 'Aadhar-data'
# df1  = pd.read_csv('Aadhar-data/HmaariCopy-Pro_Anti Policy Annotation - Aadhar-Debanjan.csv')[['Pro/Anti/Neutral','Pure Statement']]
# df2  = pd.read_csv('Aadhar-data/HmaariCopy-Pro_Anti Policy Annotation - Aadhar-Deepak.csv')[['Pro/Anti/Neutral','Pure Statement']]
# df3  = pd.read_csv('Aadhar-data/HmaariCopy-Pro_Anti Policy Annotation - Aadhar-Kapil.csv')[['Pro/Anti/Neutral','Pure Statement']]
# df = pd.concat([df1,df2,df3]).dropna()
# df.columns = ['target', 'statement']
# df.to_csv('Aadhar-data/Aadhar-data.csv')

Combined Policy Dataset

In [5]:
# filenames = listdir('../data/new')
# files = [ filename for filename in filenames if filename.endswith('.csv') ]
# df = pd.read_csv('data/'+files[0])[['Pro/Anti/Neutral','Pure Statement']]
# df.columns = ['target', 'statement']
# for file in files[1:]:
#     currdf = pd.read_csv('data/'+file)[['Pro/Anti/Neutral','Pure Statement']]
#     currdf.columns = ['target', 'statement']
#     df = pd.concat([df,currdf]).dropna()
# df.to_csv('data/combined-data.csv')

In [6]:
# df = pd.read_csv('data/combined-data.csv')
# df.groupby('target').nunique()

In [7]:
# def correctlabels(elem):
#     if elem=='anti':
#         elem = 'Anti'
#     elif elem=='pro':
#         elem='Pro'
#     elif elem=='NEutral':
#         elem= 'Neutral'   
#     return elem

In [8]:
# df['target'] = df['target'].apply(lambda x: correctlabels(x))

In [9]:
# df = df[df.target != '???']
# df = df[df.target != 'NOt A STATEMENT']
# df.groupby('target').nunique()

In [10]:
# le = LabelEncoder()
# le.fit((list(df['target'].unique())))
# df['y'] = le.transform(df['target'])
# df = df[['target','statement','y']]

In [11]:
# df.head()

## Prepare Corpus

### Combine all articles

### economic - all

In [12]:
corpus_path = "../data/lm/"
economic_path = "../data/lm/economic/"
economic_files = []
for root, dirs, files in os.walk(economic_path):
    for file in files:
        if file.endswith(".txt"):
             economic_files.append(os.path.join(root,file))

### economic - only three

In [13]:
# corpus_path = "../data/lm/"
# economic_files = []

# economic_path = "../data/lm/economic/aadhar"
# for root, dirs, files in os.walk(economic_path):
#     for file in files:
#         if file.endswith(".txt"):
#              economic_files.append(os.path.join(root,file))

# economic_path = "../data/lm/economic/demon"
# for root, dirs, files in os.walk(economic_path):
#     for file in files:
#         if file.endswith(".txt"):
#              economic_files.append(os.path.join(root,file))

# economic_path = "../data/lm/economic/farmers"
# for root, dirs, files in os.walk(economic_path):
#     for file in files:
#         if file.endswith(".txt"):
#              economic_files.append(os.path.join(root,file))

### tech

In [14]:
tech_path = "../data/lm/tech/"
tech_files = []
for root, dirs, files in os.walk(tech_path):
    for file in files:
        if file.endswith(".txt"):
             tech_files.append(os.path.join(root,file))

In [15]:
all_economic_articles = os.path.join(corpus_path, "all_economic_articles.txt")
all_tech_articles = os.path.join(corpus_path, "all_tech_articles.txt")
all_economic_articles, all_tech_articles

('../data/lm/all_economic_articles.txt', '../data/lm/all_tech_articles.txt')

In [7]:
# three_economic_articles = os.path.join(corpus_path, "three_economic_articles.txt")

In [16]:
with open(all_economic_articles, 'w') as outfile:
    for fname in economic_files:
        with open(fname) as infile:
            outfile.write(infile.read())

with open(all_tech_articles, 'w') as outfile:
    for fname in tech_files:
        with open(fname) as infile:
            outfile.write(infile.read())

In [17]:
# with open(three_economic_articles, 'a') as outfile:
#     for fname in economic_files:
#         print(fname)
#         with open(fname, 'r') as infile:
#             for line in infile:
#                 outfile.write(line)

In [18]:
# with open('../data/lm/three-articles-economic.txt', 'a') as append_to_file, open('../data/lm/economic_articles.txt', 'r') as in_file:
#         for line in in_file:
#             append_to_file.write(line)

In [19]:
with open('../data/lm/all-articles-economic.txt', 'a') as append_to_file, open('../data/lm/economic_articles.txt', 'r') as in_file:
        for line in in_file:
            append_to_file.write(line)

### Label Entities: NER

In [20]:
def replace_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,ent.label_)
    return sent

In [21]:
def blackout_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent = sent.replace(ent.text,"")
    return sent

In [22]:
def replace_and_blackout_entity(sent):
    doc = nlp(sent)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            sent1 = sent.replace(ent.text,ent.label_)
            sent2 = sent.replace(ent.text,"")
    return sent1, sent2

In [23]:
s = 'European authorities fined Google a record $5.1 billion\
    on Wednesday for abusing its power in the mobile phonemarket\
    and ordered the company to alter its practices'
# s = sentences[0]+'$$$'+sentences[1]
doc = nlp(s)
print([(X.text, X.label_) for X in doc.ents])
print(replace_entity(s))

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]
European authorities fined Google a record $5.1 billion    on Wednesday for abusing its power in the mobile phonemarket    and ordered the company to alter its practices


In [24]:
s = 'Finance Minister Arun Jaitley said on Thursday the Rs 4,500 allowance had been misused over the past week, and declared the government could not tolerate it any more.;;Earlier in the day, Economic Affairs Secretary Shaktikant Das said the lowering of the currency exchange limit was intended to enable a larger number of people to reach bank counters.;;Who benefitsFarmers with crop loans and kisan credit cards can withdraw up to Rs 25,000 a week.'
# s = sentences[0]+'$$$'+sentences[1]
doc = nlp(s)
print([(X.text, X.label_) for X in doc.ents])
print(blackout_entity(s))

[('Arun Jaitley', 'PERSON'), ('Thursday', 'DATE'), ('Rs', 'ORG'), ('4,500', 'CARDINAL'), ('the past week', 'DATE'), ('the day', 'DATE'), ('Economic Affairs', 'ORG'), ('Shaktikant Das', 'PERSON'), ('Rs', 'GPE'), ('25,000', 'CARDINAL')]
Finance Minister  said on Thursday the Rs 4,500 allowance had been misused over the past week, and declared the government could not tolerate it any more.;;Earlier in the day, Economic Affairs Secretary  said the lowering of the currency exchange limit was intended to enable a larger number of people to reach bank counters.;;Who benefitsFarmers with crop loans and kisan credit cards can withdraw up to Rs 25,000 a week.


In [25]:
displacy.render(doc, jupyter=True, style='ent')

In [26]:
displacy.render(nlp('At UIDAI, we are very strict on the privacy issue'), jupyter=True, style='dep', options={'distance':100})

In [27]:
lock_factor=0.9
window_size = 3
num_neg_samples = 10
num_epochs = 5
learning_rate = 0.001
min_count=1
pretrained_embeddings_path = '/MTP/finetune-word2vec/w2v-models/GoogleNews-vectors-negative300.bin'

In [28]:
model_path = '/MTP/w2v-models/economic-word2vec-ner'
# model_path = '/MTP/w2v-models/economic-word2vec-blackout'
policy_articles_path = all_economic_articles

In [29]:
# model_path = '/MTP/w2v-models/tech-word2vec-ner'
# model_path = '/MTP/w2v-models/tech-word2vec-blackout'
# policy_articles_path = all_tech_articles

In [30]:
with open(policy_articles_path,'r') as f:
    sentences = f.readlines()

In [31]:
with open(policy_articles_path+'-ner.txt','w') as f:
    for i, sent in tqdm_notebook(enumerate(sentences)):
        f.write(replace_entity(sent))

In [32]:
with open(policy_articles_path+'-blackout.txt','w') as f:
    for i, sent in tqdm_notebook(enumerate(sentences)):
        f.write(replace_entity(sent))

In [ ]:
# with open(policy_articles_path+'-ner.txt','w') as f1, open(policy_articles_path+'-blackout.txt','w') as f2:
#     for i, sent in tqdm_notebook(enumerate(sentences)):
#         s1, s2 = replace_and_blackout_entity(sent)
#         f1.write(s1)
#         f2.write(s2)